In [74]:
import sys
import numpy as np
import pandas as pd
sys.path.append('..')
from sklearn.decomposition import PCA
from unbalanced_dataset import UnderSampler, NearMiss, CondensedNearestNeighbour

In [75]:
from utils.data_loading import load_users_data
train_users, test_users = load_users_data()

labels = train_users['country_destination'].values
train_users = train_users.drop(['country_destination'], axis=1)

id_test = test_users['id']
piv_train = train_users.shape[0]

users = pd.concat((train_users, test_users), axis=0, ignore_index=True)
users = users.drop(['id', 'date_first_booking'], axis=1)

In [76]:
# Fill NaN values
users = users.fillna(-1)

In [77]:
import datetime
users['date_account_created'] = pd.to_datetime(users['date_account_created'])
users['year_account_created'] = pd.DatetimeIndex(users['date_account_created']).year
users['month_account_created'] = pd.DatetimeIndex(users['date_account_created']).month
users['day_account_created'] = pd.DatetimeIndex(users['date_account_created']).day
users = users.drop(['date_account_created'], axis=1)

users['timestamp_first_active'] = pd.to_datetime(users['timestamp_first_active'], format='%Y%m%d%H%M%S')
users['year_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).year
users['month_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).month
users['day_first_active'] = pd.DatetimeIndex(users['timestamp_first_active']).day
users = users.drop(['timestamp_first_active'], axis=1)

age_values = users.age.values
users['age'] = np.where(np.logical_or(age_values < 14, age_values > 100), -1, age_values)

from utils.preprocessing import one_hot_encoding

categorical_features = [
    'gender', 'signup_method', 'signup_flow', 'language', 'affiliate_channel',
    'affiliate_provider', 'first_affiliate_tracked', 'signup_app',
    'first_device_type', 'first_browser'
]

users = one_hot_encoding(users, categorical_features)

from sklearn.preprocessing import LabelEncoder

# Splitting train and test
values = users.values
X = values[:piv_train]
le = LabelEncoder()
y = le.fit_transform(labels)
X_test = values[piv_train:]

In [78]:
pca = PCA(n_components = 1)

In [79]:
x_vis = pca.fit_transform(X)

In [80]:
US = UnderSampler()
usx, usy = US.fit_transform(X, y)

Determining classes statistics... 12 classes detected: {0: 539, 1: 1428, 2: 1061, 3: 2249, 4: 5023, 5: 2324, 6: 2835, 7: 124543, 8: 762, 9: 217, 10: 62376, 11: 10094}
Under-sampling performed: Counter({0: 217, 1: 217, 2: 217, 3: 217, 4: 217, 5: 217, 6: 217, 7: 217, 8: 217, 9: 217, 10: 217, 11: 217})


In [81]:
# Generate the new dataset using under-sampling method
verbose = False
ratio = float(np.count_nonzero(y==1)) / float(np.count_nonzero(y==0))

In [85]:
# 'SMOTE Tomek links'
STK = SMOTEENN(k=1)
STK.fit_transform(X, y)

Determining classes statistics... 12 classes detected: {0: 539, 1: 1428, 2: 1061, 3: 2249, 4: 5023, 5: 2324, 6: 2835, 7: 124543, 8: 762, 9: 217, 10: 62376, 11: 10094}
Generated 217 new samples ...


ValueError: all the input arrays must have same number of dimensions

In [ ]:
pd.Series(stky).value_counts()